Given a Run and a set of events of this Run, this NB searches and saves them. Only S2 sipmt data and pmt total energy data is saved. Inputs needed are:

- Run number.
- Events number to select.
- The number of the last partition in the Run that you want to search.
- Directory where pmaps files are saved.

In [1]:
import os

import invisible_cities.io.pmaps_io as pm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#INPUTS:

run       = 6202
npart     = 8000  #number of the last partition, or higher
directory = "/home/gonzalo/Documentos/NEXT/DATOS/{}/pmaps".format(run)

n_ev  = 1000 #number of events in each file

In [3]:
#EVENTS TO SELECT (choose one)

#cs
cs_evt = np.load("cs_evts_{}.npz".format(run))['A_evtnum']

#descape
descape_evt = np.load("descape_evts_{}.npz".format(run))['A_evtnum']

#th
th_evt = np.load("photopeakTh_evts_{}.npz".format(run))['A_evtnum']

evts = th_evt
evts.sort()     #events sorting (this is not compulsory)

In [4]:
#evts.sort()
#evts_part  = [evts[n_ev*i:(i+1)*n_ev] for i in range(0, int(len(evts)/n_ev))] + [evts[n_ev * int(len(evts)/n_ev):]]
#partitions = ["{:04}".format(i) for i in range(0, npart+1)]

In [4]:
#SEARCHER AND DATAFRAME CREATOR

#COMMENT: do not mistake the number of partition of pmap, with the number of chunk of the selected data

part = 0
n_chunk = int(len(evts)/n_ev) + 1 #total number of chunk

for j in range(0, n_chunk): #j is the number of chunk
    file = 'Thphotopeak_{}_{}.h5'.format(run, j) #filename of the saved data
    i = 0
    partitions = ["{:04}".format(i) for i in range(int(part), npart+1)]
    for part in partitions:
        try:
            #partition data as dataframe
            filename = "{}/pmaps_{}_{}_v0.9.7-19-g574c657_20180710_krth1300.h5".format(directory, part, run)
            s1, s2, s2si, s1pmt, s2pmt = pm.load_pmaps_as_df(filename)
            
            #event search
            ar1 = np.unique(s1['event'])
            coinc = np.intersect1d(ar1, evts, True) #coincident events
        
            for event in coinc:
                s1_ev   = s1[s1.event == event]
                s2_ev   = s2   [s2.event    == event]
                s2si_ev = s2si [s2si.event  == event]
                s1pmt_ev= s1pmt[s1pmt.event == event]
                s2pmt_ev= s2pmt[s2pmt.event == event]
            
                s1_ev   .to_hdf(file, key = 's1'   , append = True)
                s2_ev   .to_hdf(file, key = 's2'   , append = True)
                s2si_ev .to_hdf(file, key = 's2si' , append = True)
                s1pmt_ev.to_hdf(file, key = 's1pmt', append = True)
                s2pmt_ev.to_hdf(file, key = 's2pmt', append = True)
                
                #this is useful if some of the coinc should be saved in a diferent chunk
                evts = np.delete(evts, np.argwhere(evts == event))
                
                i+=1
                print('Events saved in chunk {:02} of {:02}: {:03} %'.format(j, n_chunk-1
                                                                              , int(i*100/n_ev)), end = '\r')
                if i >= n_ev: break
                    
            if i>= n_ev: break
         
        except OSError: continue
    if len(evts)==0: break

In [6]:
#READ (for comprobation purposes)
hdf = pd.HDFStore(file)

In [7]:
df = hdf['s2si']

In [8]:
len(events)

457

In [9]:
len(df)

28456517